In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
def load_as_list(fname):
    df = pd.read_csv(fname)
    id = df['id'].values.tolist()
    label = df['label'].values.tolist()
    tweets = df['tweet'].values.tolist()
    return tweets, label

tweets, label = load_as_list("train.csv")
print(tweets[:5])
# vocab = list(set(tweets))
for i in range(len(tweets)):
    tweets[i] = ' '.join([word for word in tweets[i].split() if word not in cachedStopWords])

print(len(tweets))
print(len(label))
# print(len(vocab))
print(tweets[:5])
# print(vocab[:5])

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run', "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked", '  bihday your majesty', '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ', ' factsguide: society now    #motivation']
31962
31962
['@user father dysfunctional selfish drags kids dysfunction. #run', "@user @user thanks #lyft credit can't use cause offer wheelchair vans pdx. #disapointed #getthanked", 'bihday majesty', '#model love u take u time urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91 ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦', 'factsguide: society #motivation']


In [3]:
word_count = {}
for word in tweets:
    word_list = word.split()
    for sub_word in word_list:
        if sub_word in word_count:
            word_count[sub_word] += 1
        else:
            word_count[sub_word] = 1
print("executed")

word_count = {key:val for key, val in word_count.items() if val >= 30}
# print(word_count)
print(len(word_count))

executed
1197


In [4]:
accepted_list = list(word_count.keys())

for word in tweets:
    word_list = word.split()
    final_word_list = [word for word in word_list if word in accepted_list]
    word = ' '.join(final_word_list)
    
print(tweets[:5])
print(len(tweets))
print(len(label))
    

['@user father dysfunctional selfish drags kids dysfunction. #run', "@user @user thanks #lyft credit can't use cause offer wheelchair vans pdx. #disapointed #getthanked", 'bihday majesty', '#model love u take u time urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91 ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦', 'factsguide: society #motivation']
31962
31962


In [5]:
vocab_size = 1200
encoded_docs = [one_hot(d, vocab_size) for d in tweets]
print(encoded_docs[:5])

# pad documents to a max length of 4 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

[[1132, 40, 135, 183, 126, 39, 539, 499], [1132, 1132, 816, 1109, 912, 626, 136, 715, 248, 1122, 903, 260, 817, 1182], [728, 76], [1038, 1104, 715, 404, 715, 939, 1181, 566, 59], [459, 154, 403]]
31962


In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(LSTM(1024))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, label, epochs=100, batch_size = 100)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           38400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              4329472   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 4,368,897
Trainable params: 4,368,897
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\Amaan_Hussain\anaconda3\envs\keras_env\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
 2000/31962 [>.............................] - ETA: 1:52:51 - loss: 0.3984 - accuracy: 0.9315